In [ ]:
import time
import copy
import logging
import networkx
from networkx import convert_matrix
from typing import NamedTuple, Optional, Tuple, List, Dict, TypeVar, TypedDict, cast

import numpy
import gym
from gym import spaces
from gym.utils import seeding

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from cyberbattle._env.shared_cyberbattle_env import EnvironmentBounds, AttackerGoal, DefenderGoal, DefenderConstraint
from cyberbattle._env.defender import DefenderAgent,ScanAndReimageCompromisedMachines
from cyberbattle.simulation.model import PortName, PrivilegeLevel
from cyberbattle.simulation import commandcontrol, model, actions
from cyberbattle._env.discriminatedunion import DiscriminatedUnion
from cyberbattle.agents.baseline import agent_wrapper as w
import cyberbattle.agents.baseline.learner as learner
from cyberbattle.agents.baseline.learner import Learner
import cyberbattle.agents.baseline.agent_tabularqlearning as a
import cyberbattle.agents.baseline.agent_ddql as ddqla
import cyberbattle.agents.baseline.agent_dql as dqla
from cyberbattle.defender_agents.trainedDefender import *
import random
random.seed(120394016)
%matplotlib inline
import cyberbattle.agents.baseline.agent_tabularqlearning as a
import cyberbattle.agents.baseline.learner as learner
from cyberbattle.agents.baseline.agent_wrapper import Verbosity
#ql
iteration_count = 10
training_episode_count = 10
eval_episode_count = 20
gamma_sweep = [
    0.015,  # about right
]
import gym

gym_env = gym.make('CyberBattleChain-v0',
    size=12,
    attacker_goal=AttackerGoal(
        own_atleast=0,
        own_atleast_percent=1.0
    ),
    defender_constraint=DefenderConstraint(
        maintain_sla=0.80
    )
)

defender_name = "DefenderName"
defender_agent = DefenderWrapper(gym_env,defender_name)
from cyberbattle.agents.baseline.agent_wrapper import ActionTrackingStateAugmentation, StateAugmentation
from cyberbattle.agents.baseline.plotting import PlotTraining, plot_averaged_cummulative_rewards
import progressbar
import math
from cyberbattle.agents.baseline import learner
import sys
from cyberbattle.simulation.model import FirewallRule
ep = w.EnvironmentBounds.of_identifiers(
    maximum_total_credentials=22,
    maximum_node_count=22,
    identifiers=gym_env.identifiers
)
#cyberbattlechain_defender.render_as_fig()
import cyberbattle.agents.baseline.agent_tabularqlearning as a
import cyberbattle.agents.baseline.learner as learner
from cyberbattle.agents.baseline.agent_wrapper import Verbosity
epsilon = 0.66666666666666
gamma= 0.06925
learning_rate= 0.34444444444444444
epsilon_exponential_decay=3198.830407105263
epsilon_minimum=0.05708811937931034
episode_count=10
iteration_count=10
epsilon_multdecay=0.7777777777777778
statespace = w.HashEncoding(ep, [
            # Feature_discovered_node_count(),
            # Feature_discovered_credential_count(),
            w.Feature_discovered_ports_sliding(ep),
            w.Feature_discovered_nodeproperties_sliding(ep),
            w.Feature_discovered_notowned_node_count(ep, 3)
        ], 5000)  # should not be too small, pick something big to avoid collision

actionspace = w.RavelEncoding(ep, [
            w.Feature_active_node_properties(ep)])
print(statespace)
print(actionspace)
q_matrix = DefenderQMatrix(statespace, actionspace,gym_env)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

class SimulationTracker:
    def __init__(self):
        self.attacker_rewards = []
        self.defender_rewards = []
        self.actions_taken = {'attacker': [], 'defender': []}
        self.exploration_actions = {'attacker': 0, 'defender': 0}
        self.exploitation_actions = {'attacker': 0, 'defender': 0}
        self.episode_lengths = []
        self.reward_variability = []
        self.state_visitation_frequencies = {}
        self.temporal_differences = []
        self.infected_nodes = []

    def log_episode(self, attacker_reward, defender_reward, episode_length, episode_rewards):
        self.attacker_rewards.append(attacker_reward)
        self.defender_rewards.append(defender_reward)
        self.episode_lengths.append(episode_length)
        self.reward_variability.append(np.var(episode_rewards))
        #self.temporal_differences.append(temporal_difference)

    def log_action(self, role, action, exploration):
        action_str = str(action)  # Convert the action dictionary to a string
        self.actions_taken[role].append(action_str)
        if exploration:
            self.exploration_actions[role] += 1
        else:
            self.exploitation_actions[role] += 1


    def log_state_visitation(self, state):
        if state not in self.state_visitation_frequencies:
            self.state_visitation_frequencies[state] = 1
        else:
            self.state_visitation_frequencies[state] += 1

    def log_infected_nodes(self, nodes_count):
        self.infected_nodes.append(nodes_count)

    def print_summary(self):
        # Summary statistics
        print("Simulation Summary:")
        print(f"Average Attacker Reward: {np.mean(self.attacker_rewards)}")
        print(f"Average Defender Reward: {np.mean(self.defender_rewards)}")
        print(f"Average Episode Length: {np.mean(self.episode_lengths)} steps")
        print(f"Average Reward Variability: {np.mean(self.reward_variability)}")
        print(f"Average Temporal Difference: {np.mean(self.temporal_differences)}")
        print(f"Average Infected Nodes per Episode: {np.mean(self.infected_nodes)}")

        #Exploration vs. Exploitation
        print("\nExploration vs. Exploitation:")
        for role in ['attacker', 'defender']:
            total_actions = self.exploration_actions[role] + self.exploitation_actions[role]
            if total_actions > 0:
                print(f"{role.capitalize()} Exploration: {self.exploration_actions[role] / total_actions:.2f}")
                print(f"{role.capitalize()} Exploitation: {self.exploitation_actions[role] / total_actions:.2f}")
        self._print_action_distributions()
        self._plot_rewards()

    def _print_action_distributions(self):
        print("\nAction Distributions:")
        for role, actions in self.actions_taken.items():
            # actions are already converted to strings by log_action
            unique_actions, counts = np.unique(actions, return_counts=True)
            print(f"{role.capitalize()} Actions:")
            for action, count in zip(unique_actions, counts):
                print(f"Action {action}: {count} times")


    def _plot_rewards(self):
        plt.figure(figsize=(10, 5))
        plt.plot(self.attacker_rewards, label='Attacker Rewards')
        plt.plot(self.defender_rewards, label='Defender Rewards')
        plt.xlabel('Episode')
        plt.ylabel('Reward')
        plt.title('Rewards per Episode for Attacker and Defender')
        plt.legend()
        plt.show()


# train attacker

In [ ]:
random.seed(120394016)
# Parameter ranges
epsilon = 0.9629629629629622
#gamma=np.linspace(0.05,0.09,10)
gamma= 0.05888888888888889
#learning_rate=np.linspace(0.3,.35,10)
learning_rate=0.3222222222222222
#epsilon_decay=np.linspace(2333.33333,5000,20)
epsilon_exponential_decay=5000
#epsilon_minimum=np.linspace(0.01,.07777777,10)
epsilon_minimum=0.0513409942068965
episode_count=50
iteration_count=200
#epsilon_multdecay=np.linspace(0.3,0.875,20)
epsilon_multdecay=0.6631578947368421

trained_attacker = learner.epsilon_greedy_search(
        gym_env,
        ep,
        a.QTabularLearner(ep, gamma=0.05888888888888889, learning_rate=0.3222222222222222, exploit_percentile=100),
        episode_count=episode_count,
        iteration_count=iteration_count,
        epsilon=epsilon,
        render=True,
        epsilon_multdecay=epsilon_multdecay,  # 0.999,
        epsilon_exponential_decay=epsilon_exponential_decay,
        epsilon_minimum=epsilon_minimum,
        verbosity=Verbosity.Quiet,
        title="Q-learning"
    )

In [ ]:
import cyberbattle.agents.baseline.agent_tabularqlearning as a
import cyberbattle.agents.baseline.learner as learner
from cyberbattle.agents.baseline.agent_wrapper import Verbosity
epsilon = 0.666666
gamma= 0.06925
learning_rate= 0.34444444444444444
epsilon_exponential_decay=3198.830407105263
epsilon_minimum=0.05708811937931034
episode_count=50
iteration_count=200
epsilon_multdecay=0.7777777777777778
statespace = w.HashEncoding(ep, [
            # Feature_discovered_node_count(),
            # Feature_discovered_credential_count(),
            w.Feature_discovered_ports_sliding(ep),
            w.Feature_discovered_nodeproperties_sliding(ep),
            w.Feature_discovered_notowned_node_count(ep, 3)
        ], 5000)  # should not be too small, pick something big to avoid collision

actionspace = w.RavelEncoding(ep, [
            w.Feature_active_node_properties(ep)])
print(statespace)
print(actionspace)
q_matrix = DefenderQMatrix(statespace, actionspace,gym_env)
learner2 = DefenderQLearner(gym_env, defender_agent, ep, q_matrix, epsilon=0.9, gamma=0.015, learning_rate=.07)
random.seed(120394016)
#%matplotlib inline

trained_defender = epsilon_greedy_defender_training(
        cyberbattle_gym_env=gym_env,
        environment_properties=ep,
        learner=learner2,
        episode_count=episode_count,  # Set the number of episodes for the training
        iteration_count=iteration_count,  # Set the number of iterations per episode
        epsilon=epsilon,
        epsilon_exponential_decay=epsilon_exponential_decay,
        epsilon_minimum=epsilon_minimum,
        epsilon_multdecay=epsilon_multdecay,
        verbosity=logging.INFO,
        render=True,
        plot_episodes_length=True,
        title="Defender Q-Learning Training"
    )

In [ ]:
import gym

gym_env = gym.make('CyberBattleChain-v0',
    size=12,
    attacker_goal=AttackerGoal(
        own_atleast=0,
        own_atleast_percent=1.0
    ),
    defender_constraint=DefenderConstraint(
        maintain_sla=0.80
    )
)

defender_name = "DefenderName" 
defender_agent = DefenderWrapper(gym_env,defender_name)

gym_environment = gym.make('CyberBattleChain-v0',
    size=12,
    attacker_goal=AttackerGoal(
        own_atleast=0,
        own_atleast_percent=1.0
    ),
    defender_constraint=DefenderConstraint(
        maintain_sla=0.80
    ),
    defender_agent=defender_agent
)

#this was made due to issues where the attacker q learner needed to use a wrapper environment
from cyberbattle.agents.baseline.agent_wrapper import AgentWrapper, StateAugmentation
def wrap_environment(env):
    initial_observation = env.reset()  
    state = StateAugmentation(initial_observation)  
    return AgentWrapper(env, state)

attacker_learner = trained_attacker['learner']
defender_learner = trained_defender['learner']
import numpy as np
import matplotlib.pyplot as plt

# Simulation Parameters
number_of_episodes = 50
max_steps_per_episode = 200
attacker_epsilon = 0.1
defender_epsilon = 0.1

# Reward Tracking
attacker_rewards_per_episode = []
defender_rewards_per_episode = []
win_rates = {'attacker': [], 'defender': []}
actions_taken_count = {'attacker': [], 'defender': []}
exploration_vs_exploitation = {'exploration': [], 'exploitation': []}
episode_lengths = []
state_visitation_frequencies = {}  
reward_variability = []
defender_successful_actions = []
defender_failed_actions = []
defender_actions_taken = []
tracker = SimulationTracker() #instantiate simulation tracker
# Assuming wrap_environment and gym_environment are defined elsewhere
wrapped_env = wrap_environment(gym_environment)
win_counts = {'attacker': 0, 'defender': 0}
# Main Simulation Loop
random.seed(120394016)
for episode in range(number_of_episodes):
    observation = wrapped_env.reset()
    attacker_total_reward = 0
    defender_total_reward = 0
    steps_this_episode=0
    episode_rewards = []  # Track rewards for this episode
    for step in range(max_steps_per_episode):
        steps_this_episode +=1
        # Attacker's Turn
        exploration = np.random.random() < attacker_epsilon
        if exploration:
            _, attacker_action, _ = attacker_learner.explore(wrapped_env)
        else:
            _, attacker_action, _ = attacker_learner.exploit(wrapped_env, observation)
        #print(attacker_action)
        if attacker_action is not None:
            observation, attacker_reward, done, _ = gym_environment.step(attacker_action)
            attacker_total_reward += attacker_reward
            print(_)
            actions_taken_count['attacker'].append(attacker_action)  # Track action
            episode_rewards.append(attacker_reward)  # Track reward variability
            tracker.log_action('attacker', attacker_action, exploration)
            #actions_taken_count['attacker'][attacker_action] = actions_taken_count['attacker'].get(attacker_action, 0) + 1


        # Defender's Turn
        exploration = np.random.random() < defender_epsilon
        if exploration:
            _, defender_action, _ = defender_learner.explore(wrapped_env)
        else:
            _, defender_action, _ = defender_learner.exploit(wrapped_env, observation)

        if defender_action is not None:
            observation, defender_reward, done, _ = gym_environment.step(defender_action)
            #action= _['defender_info']['action']
            defender_total_reward += defender_reward
            actions_taken_count['defender'].append(defender_action)  # Track action
            episode_rewards.append(defender_reward)  # Track reward variability
            tracker.log_action('defender', defender_action, exploration)
            #actions_taken_count['defender'][defender_action] = actions_taken_count['defender'].get(defender_action, 0) + 1
        #logging
        # After each episode
        tracker.log_episode(attacker_total_reward, defender_total_reward, steps_this_episode, episode_rewards)
        tracker.print_summary()
        tracker._plot_rewards()

        if done:
            break
            
    defender_info = _['defender_info']  
    if defender_info:  
        defender_actions_taken.append(defender_info['performance_metrics']['actions_taken'])
        defender_successful_actions.append(defender_info['performance_metrics']['successful_actions'])
        defender_failed_actions.append(defender_info['performance_metrics']['failed_actions'])

    attacker_rewards_per_episode.append(attacker_total_reward)
    defender_rewards_per_episode.append(defender_total_reward)
    episode_lengths.append(steps_this_episode)
    reward_variability.append(np.var(episode_rewards))
    print(f"Episode {episode + 1}: Attacker Reward = {attacker_total_reward}, Defender Reward = {defender_total_reward}")

wrapped_env.close()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

episodes = np.arange(1, number_of_episodes + 1)

plt.figure(figsize=(14, 7))
plt.plot(episodes, attacker_rewards_per_episode, label='Attacker Rewards', color='blue', linestyle='-', marker='o')
plt.plot(episodes, defender_rewards_per_episode, label='Defender Rewards', color='red', linestyle='-', marker='x')
plt.title('Rewards per Episode for Attacker and Defender')
plt.xlabel('Episode')
plt.ylabel('Total Reward')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(14, 7))
plt.bar(episodes, defender_successful_actions, color='lightblue')
plt.title('Successful Defender Actions Per Episode')
plt.xlabel('Episode')
plt.ylabel('Number of Successful Actions')
plt.tight_layout()
plt.show()
